# Splits extraction using dedicated service - Python

### Overview
This sample requests **on-demand** splits by calling a dedicated data service.

### Inputs/outputs
Splits sample is designed to help you searching and requesting dividends for a given instrument.
* It takes instrument specification parameter(s) as per input filter and returns *Dividends*

### Services used
This sample uses *gRPC requests* in order to retrieve splits data from the hosted service. The queried endpoint in this script are:
* *SplitsService*: to directly retrieve splits data objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.corporate_actions.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run splits queries

### Step 1: Install packages and import them

In [1]:
pip install googleapis-common-protos protobuf grpcio pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install systemathics.apis

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.corporate_actions.v1.splits_pb2 as splits
import systemathics.apis.services.corporate_actions.v1.splits_pb2_grpc as splits_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [4]:
token = token_helpers.get_token()
display(token)

'Bearer eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IlpBV05FbDJfMF9zTDNUTklIdlN3SiJ9.eyJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9lbWFpbGFkZHJlc3MiOiJvY3RhdmUuYmVsb3RAc3lzdGVtYXRoaWNzLmNvbSIsImlzcyI6Imh0dHBzOi8vZ2FueW1lZGUtZGV2LmV1LmF1dGgwLmNvbS8iLCJzdWIiOiJhdXRoMHw2MjgxZjk3ZmM0ZWE4ZjAwNjkwMDUyMDMiLCJhdWQiOlsiaHR0cHM6Ly9kZXYuZ2FueW1lZGUtZGV2IiwiaHR0cHM6Ly9nYW55bWVkZS1kZXYuZXUuYXV0aDAuY29tL3VzZXJpbmZvIl0sImlhdCI6MTY1MjY4NjY5OSwiZXhwIjoxNjU1Mjc4Njk5LCJhenAiOiJrTnNhak50d3ZyblU1bVZCa0c5ZnZIbVdNU1NJMFN5TyIsInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZW1haWwiLCJwZXJtaXNzaW9ucyI6WyJhZG1pbiIsImFydGVmYWN0cyIsImNoYW5nZXMiLCJjb21wb25lbnRzIiwiZGFpbHlfYmFycyIsImRhaWx5X2JvbGxpbmdlciIsImRhaWx5X2NtYSIsImRhaWx5X2VtYSIsImRhaWx5X21hY2QiLCJkYWlseV9wcmljZXMiLCJkYWlseV9yc2kiLCJkYWlseV9zbWEiLCJkYWlseV92b2xhdGlsaXR5IiwiZGFpbHlfdndhcHMiLCJkaXZpZGVuZHMiLCJpZGVudGlmaWVyX21hcHBpbmciLCJpbnRyYWRheV9iYXJzIiwiaW50cmFkYXlfYm9sbGluZ2VyIiwiaW50cmFkYXlfY21hIiwiaW50cmFkYXlfZW1hIiwiaW50cmFkYXlfbWFjZCIsIml

### Step 3: Retrieve splits

#### 3.1 Equity selection

In [5]:
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 Create and process request

In [6]:
# create the splits request
request = splits.SplitsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
)

In [7]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        #instantiate the splits service
        service = splits_service.SplitsServiceStub(channel)
        
        # process the request
        response = service.Splits(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### 3.3 Retrieve splits

In [8]:
# prepare the splits data frame content
dates = [datetime(d.date.year, d.date.month, d.date.day) for d in response.data]
old_shares = [d.old_shares for d in response.data]
new_shares = [d.new_shares for d in response.data]

# create pandas dataframe
d = {'Date': dates, 'New shares': new_shares, 'Old shares': old_shares}
df = pd.DataFrame(data=d)

In [9]:
# display splits
df

,Date,New shares,Old shares
0,2014-06-09,1.0,7.0
1,2020-08-31,1.0,4.0


#### 3.4 Add dates constraints

In [10]:
# set data interval (we are using Google date time format) 
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2020, month = 1, day = 1),
    end_date = date.Date(year = 2021, month = 3, day = 31)
)

# generate constraints based on the previous date selection
# time constraints are not neeeded for the dividends
my_constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

In [11]:
# create the splits request with dates constraints
request = splits.SplitsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = my_constraints
)

In [12]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        #instantiate the dividends service
        service = splits_service.SplitsServiceStub(channel)
        
        # process the request
        response = service.Splits(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### 3.5 Retrieve splits with date constraints

In [13]:
# prepare the splits data frame content
dates = [datetime(d.date.year, d.date.month, d.date.day) for d in response.data]
old_shares = [d.old_shares for d in response.data]
new_shares = [d.new_shares for d in response.data]

# create pandas dataframe
d = {'Date': dates, 'New shares': new_shares, 'Old shares': old_shares}
df = pd.DataFrame(data=d)

In [14]:
# display splits
df

,Date,New shares,Old shares
0,2020-08-31,1.0,4.0
